# Going through data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('./data/bycountry_ds.json')
df.shape

(17403, 5)

In [3]:
df.head()

,category,input,output_low,output_medium,output_high
http://dbpedia.org/resource/...All_the_Marbles,[United States],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0
http://dbpedia.org/resource/An_Eye_for_an_Eye_(1981_film),[United States],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0
http://dbpedia.org/resource/Brain_Dead_(1990_film),[United States],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0
http://dbpedia.org/resource/For_Pete's_Sake_(film),[United States],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0
http://dbpedia.org/resource/Hannah_Montana:_The_Movie,[United States],"[25, 6677, 48, 2, 138, 5, 331, 1, 7, 25, 6983,...",0,0,1


In [4]:
from tensorflow import keras

In [5]:
model = keras.models.load_model('./models/bycountry_model')

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           640000    
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 652,515
Trainable params: 652,515
Non-trainable params: 0
_________________________________________________________________


In [7]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)
    if i != 0: # Embedding does not have activation function
        print(layer.activation)

0 embedding
1 lstm_3
<function tanh at 0x000002E1F30CD820>
2 dense
<function softmax at 0x000002E1F30C8A60>


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

Building a recomposed model

In [27]:
layer_to_explore = 1

In [28]:
model2 = Sequential()

for i in range(len(model.layers)):
    if i < layer_to_explore + 1:
        print(f"Adding layer {i}")
        model2.add(model.layers[i])
        
model2.summary()

Adding layer 0
Adding layer 1
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           640000    
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
Total params: 652,416
Trainable params: 652,416
Non-trainable params: 0
_________________________________________________________________


Format for the input to be passed to model.predict

In [29]:
inputs = np.array([np.array(x) for x in df.input])

np.array(inputs).shape

(17403, 100)

In [30]:
activations = model2.predict(inputs)

544/544 [==============================] - 13s 20ms/step


In [32]:
activations[0]

0.292121

## Getting all the unique categories

In [13]:
# Need to check if the country is real
def get_unique_categories(df):
    unique_categories = []
    for i in range(len(df.category)):
        for cat in df.category[i]:
            if cat not in unique_categories and cat != '':
                unique_categories.append(cat)
                
    return unique_categories

def get_popular_categories(df, thresh=500):
    categories = get_unique_categories(df)
    
    dic = {}
    for category in categories:
        dic[category] = len(df[df.category.apply(lambda x: category in x)])
        
    return_dic = {c: n for c, n in dic.items() if n >= thresh}
    
    return sorted(return_dic.items(), key=lambda x: x[1], reverse=True) # Return sorted dictionary

In [14]:
get_unique_categories(df)

['United States',
 'France',
 'Australia',
 'United Kingdom',
 'Mexico',
 'Argentina',
 'Italy',
 'Spain',
 'United States, Mexico',
 'Belgium',
 'Germany',
 'Brazil',
 'Japan',
 'Lebanon',
 'Ireland',
 'Canada',
 'Luxembourg',
 'West Germany',
 'USA',
 'Russia',
 'China',
 'Serbia',
 'Indonesia',
 'Hong Kong',
 'Switzerland',
 'Armenia',
 'Austria',
 'India',
 'Iran',
 'Taiwan',
 'Soviet Union',
 'South Africa',
 'Romania',
 'South Korea',
 'Cambodia',
 'Netherlands',
 'Sweden',
 'Singapore',
 'Poland',
 'Norway',
 'Mali',
 'Czech Republic',
 'Denmark',
 'New Zealand',
 'Lithuania',
 'Jordan',
 'Hungary',
 'United States/Mexico',
 'Yugoslavia',
 'Italy, France',
 'Colombia',
 'Monaco',
 'Syria',
 'Slovakia',
 'Ukraine',
 'Morocco',
 'Pakistan',
 'Thailand',
 'Israel',
 'Mainland China',
 'Bulgaria',
 'Portugal',
 'Nigeria',
 'Chile',
 'Peru',
 'Iceland',
 'Algeria',
 'Tunisia',
 'Greece',
 'Malaysia',
 'Cameroon',
 'Kenya',
 'Vietnam',
 'Japan, United States',
 'Puerto Rico',
 'Finlan

In [48]:
get_popular_categories(df, thresh=300)

[('United States', 10734),
 ('United Kingdom', 1757),
 ('France', 1725),
 ('Hong Kong', 804),
 ('Australia', 592),
 ('Japan', 580),
 ('China', 567),
 ('Canada', 499),
 ('Germany', 430),
 ('India', 377),
 ('Italy', 318)]

## Getting inputs for a specific category

In [16]:
def get_inputs_for_cat(category, df):
    raw_inputs = df[df.category.apply(lambda x: category in x)].input
    inputs = []
    for i in range(len(raw_inputs)):
        inputs.append(raw_inputs[i])
    return np.array(inputs)

In [17]:
get_inputs_for_cat('France', df)

array([[   0,    0,    0, ...,  108,   23,  192],
       [   0,    0,    0, ..., 1372,  287,  121],
       [   0,    0,    0, ...,    9,  141, 2048],
       ...,
       [   0,    0,    0, ...,   22,  608, 1051],
       [   0,    0,    0, ...,   99,   24,   26],
       [   0,    0,    0, ...,  101, 3619, 1568]])

## Getting all activations and standardize the DataFrame

In [18]:
def standardize(df):
    """
    Standardize the whole dataframe
    """
    df_s = df.copy()
    for col in df:
        if "neuron" in col:
            df_s[col] = (df[col] - df[col].mean()) / df[col].std()

    return df_s

In [19]:
def get_all_activations(df, model):
    new_df = pd.DataFrame()
    
    raw_inputs = df.input
    inputs = []
    for i in range(len(raw_inputs)):
        inputs.append(raw_inputs[i])
    
    new_df['category'] = df.category
    new_df['input'] = df.input
    new_df['output_low'] = df.output_low
    new_df['output_medium'] = df.output_medium
    new_df['output_high'] = df.output_high

    activations = model.predict(inputs)
    
    if isinstance(model.layers[-1], Embedding):
        mean_activations = []
        for a in activations:
            mean_activations.append(pd.DataFrame(a).mean())
        activations = np.array(mean_activations)
    
    for neuron_index, value_list in enumerate(activations.T):
        index = f"neuron_{neuron_index + 1}"
        new_df[index] = value_list
    
    return standardize(new_df)

In [20]:
df_act = get_all_activations(df, model2)

544/544 [==============================] - 2s 3ms/step


KeyboardInterrupt: 

In [ ]:
df_act.head()

In [ ]:
c = "France"

df_s = df_act.copy()

In [ ]:
def get_cat_df(category, df):
    """
    df : The standardized dataframe
    """
    return df[df.category.apply(lambda x: category in x)]

def get_activation_for_cat(category, df):
    """
    df : The standardized dataframe
    """
    return get_cat_df(category, df).iloc[:, 5:]

In [ ]:
get_activation_for_cat('France', df_s)

In [ ]:
def get_activation_for_not_cat(category, df):
    """
    df : The standardized dataframe
    """
    return getNotCatDF(category, df).iloc[:, 5:]

def getNotCatDF(cat, df): # Standardized df
    """
    df : The standardized dataframe
    """
    return df[df.category.apply(lambda x: cat not in x)]

In [ ]:
actnc = get_activation_for_not_cat(c, df_s) # (15678, 32)

actc = get_activation_for_cat(c, df_s) # (1725, 32)

actnc.sample(len(actc))

https://likegeeks.com/seaborn-heatmap-tutorial/#Changing_heatmap_color

In [ ]:
from scipy import stats
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import time

def find_pv(category):
    start_time = time.time()
    
    actc = get_activation_for_cat(category, df_s)
    actnc = get_activation_for_not_cat(category, df_s)
    reses = []
    for i in range(1000):  
        actncs = actnc.sample(len(actc), replace=True)
        res = []
        for col in actc:
            p = stats.wilcoxon(np.array(actncs[col]), y=np.array(actc[col])).pvalue
            res.append(p)
        reses.append(res)
    return_df = pd.DataFrame(np.array(reses)).mean()
    
    print(f"--- For find_pv with c={category}: {time.time() - start_time}cseconds ---")
    return return_df

def get_heatmaps_dict(self):
    start_time = time.time()

    dheatmaps = {}

    heatmap_path = '../heatmaps/'

    if not os.path.exists(heatmap_path):
        os.makedirs(heatmap_path)

    #for i in range(len(self.model.get_layers())):
    for i in range(len(self.dfs)):
        ddf = {}

        current_path = os.path.join(heatmap_path, self.model.get_layers()[i].name)
        if not os.path.exists(current_path):
            os.makedirs(current_path)

        for c, n in self.get_popular_categories(thresh=500):
            for_cat = {}

            r = self.find_pv(c, self.dfs[i])

            # 1st heatmap
            rdf = pd.DataFrame(r)
            ax = sns.heatmap(rdf.T, cbar=False, cmap="Greys")
            fig = ax.get_figure()
            path = f"{current_path}/{c}-1.png"
            fig.savefig(path)

            for_cat['heatmap-1'] = {}
            for_cat['heatmap-1']['path'] = path
            for_cat['heatmap-1']['data'] = np.array(rdf[0]).T

            # 2nd heatmap
            rdf[0] = rdf[0].apply(lambda x: 0 if x > 0.01 else 1)
            ax = sns.heatmap(rdf.T, cbar=False, cmap="Greys")
            fig = ax.get_figure()
            path = f"{current_path}/{c}-2.png"
            fig.savefig(path)

            for_cat['heatmap-2'] = {}
            for_cat['heatmap-2']['path'] = path
            for_cat['heatmap-2']['data'] = np.array(rdf[0]).T

            # Add paths to dict
            ddf[c] = for_cat

        dheatmaps[self.model.get_layers()[i].name] = ddf

    print(f"--- For get_heatmaps_dict: {time.time() - start_time} seconds ---")

    return dheatmaps



In [ ]:
def get_heatmaps_dict():
    dheatmaps = {}
    for c, n in get_popular_categories(df_s, thresh=500):
        for_cat = {}

        r = find_pv(c)

        # 1st heatmap
        rdf = pd.DataFrame(r)
        ax = sns.heatmap(rdf.T, cbar=False, cmap="Greys")
        fig = ax.get_figure()
        path = f"../heatmaps/{c}-1.png"
        #fig.savefig(path)

        for_cat['heatmap-1'] = {}
        for_cat['heatmap-1']['path'] = path
        for_cat['heatmap-1']['data'] = np.array(rdf[0]).T

        # 2st heatmap
        rdf[0] = rdf[0].apply(lambda x: 0 if x > 0.01 else 1)
        ax = sns.heatmap(rdf.T, cbar=False, cmap="Greys")
        fig = ax.get_figure()
        path = f"../heatmaps/{c}-2.png"
        #fig.savefig(path)

        for_cat['heatmap-2'] = {}
        for_cat['heatmap-2']['path'] = path
        for_cat['heatmap-2']['data'] = np.array(rdf[0]).T

        # Add paths to dict
        dheatmaps[c] = for_cat

    return dheatmaps

In [ ]:
get_heatmaps_dict()

"""
for c, n in get_popular_categories(df):
    print(c)
    r = find_pv(c)
    rdf = pd.DataFrame(r)
    
    sns.heatmap(rdf.T, cmap="Greys")
    plt.show()
    
    """"""
    data = [go.Heatmap(z=rdf.T, zmin=0, zmax=1,
            colorscale=['rgb(255, 255, 255)', 'rgb(0, 0, 0)'], 
            reversescale=False)]
    layout = go.Layout(template='none', height=300)
    fig = go.Figure(data=data, layout=layout)
    fig.show()"""
    """
    

    rdf[0] = rdf[0].apply(lambda x: 0 if x > 0.01 else 1)
    
    sns.heatmap(rdf.T, cmap="Greys")
    plt.show()
    
    """"""
    data = [go.Heatmap(z=rdf.T, zmin=0, zmax=1,
            colorscale=['rgb(0, 0, 0)', 'rgb(255, 255, 255)'], 
            reversescale=False)]
    layout = go.Layout(template='none', height=300)
    fig = go.Figure(data=data, layout=layout)
    fig.show()
    """"""

In [ ]:
get_heatmaps_dict()

"""
for c, n in get_popular_categories(df):
    print(c)
    r = find_pv(c)
    rdf = pd.DataFrame(r)
    
    sns.heatmap(rdf.T, cmap="Greys")
    plt.show()
    
    """"""
    data = [go.Heatmap(z=rdf.T, zmin=0, zmax=1,
            colorscale=['rgb(255, 255, 255)', 'rgb(0, 0, 0)'], 
            reversescale=False)]
    layout = go.Layout(template='none', height=300)
    fig = go.Figure(data=data, layout=layout)
    fig.show()"""
    """
    

    rdf[0] = rdf[0].apply(lambda x: 0 if x > 0.01 else 1)
    
    sns.heatmap(rdf.T, cmap="Greys")
    plt.show()
    
    """"""
    data = [go.Heatmap(z=rdf.T, zmin=0, zmax=1,
            colorscale=['rgb(0, 0, 0)', 'rgb(255, 255, 255)'], 
            reversescale=False)]
    layout = go.Layout(template='none', height=300)
    fig = go.Figure(data=data, layout=layout)
    fig.show()
    """"""

In [ ]:
def get_all_activations(df, model):
    new_df = pd.DataFrame()
    
    raw_inputs = df.input
    inputs = []
    for i in range(len(raw_inputs)):
        inputs.append(raw_inputs[i])
    
    new_df['category'] = df.category
    new_df['input'] = df.input
    new_df['output_low'] = df.output_low
    new_df['output_medium'] = df.output_medium
    new_df['output_high'] = df.output_high

    activations = model.predict(inputs)
    for neuron_index, value_list in enumerate(activations.T):
        index = f"neuron_{neuron_index + 1}"
        new_df[index] = value_list
    
    return standardize(new_df)

In [21]:
raw_inputs = df.input
inputs = [x for x in df.input]

new_df = pd.DataFrame()

activations = model2.predict(inputs)
activations

544/544 [==============================] - 2s 3ms/step


array([[[ 0.03974721, -0.01969532, -0.03421203, ..., -0.0004223 ,
          0.0011404 ,  0.08591275],
        [ 0.03974721, -0.01969532, -0.03421203, ..., -0.0004223 ,
          0.0011404 ,  0.08591275],
        [ 0.03974721, -0.01969532, -0.03421203, ..., -0.0004223 ,
          0.0011404 ,  0.08591275],
        ...,
        [-0.04529711, -0.03400601,  0.01857534, ...,  0.02998969,
          0.02016949, -0.00056641],
        [-0.01474597, -0.04339802,  0.01841702, ...,  0.04422619,
         -0.07639269,  0.08313026],
        [-0.03443238, -0.0105089 ,  0.05758018, ...,  0.0113349 ,
          0.06457688,  0.0321545 ]],

       [[ 0.03974721, -0.01969532, -0.03421203, ..., -0.0004223 ,
          0.0011404 ,  0.08591275],
        [ 0.03974721, -0.01969532, -0.03421203, ..., -0.0004223 ,
          0.0011404 ,  0.08591275],
        [ 0.03974721, -0.01969532, -0.03421203, ..., -0.0004223 ,
          0.0011404 ,  0.08591275],
        ...,
        [ 0.01513298, -0.03770915, -0.03066242, ..., -

In [24]:
activations[0].mean()

-0.00063485024

In [25]:
activations.shape, activations.T.shape

((17403, 100, 64), (64, 100, 17403))

In [26]:
activations[0][0].shape

(64,)

## Testing church model

In [72]:
from keras.models import Sequential
from keras.layers import Embedding, Conv2D, MaxPooling2D

In [22]:
model = keras.models.load_model('./models/church_model')
model.layers

In [71]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn1 (Conv2D)               (None, 193, 193, 8)       1544      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 96, 8)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 73728)             0         
                                                                 
 dense (Dense)               (None, 32)                2359328   
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,360,905
Trainable params: 2,360,905
Non-trainable params: 0
______________________________________________

In [23]:
new_model = Sequential()
new_model.add(model.layers[0])

new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn1 (Conv2D)               (None, 193, 193, 8)       1544      
                                                                 
Total params: 1,544
Trainable params: 1,544
Non-trainable params: 0
_________________________________________________________________


In [24]:
data = pd.read_json('./data/church_ds.json')

In [25]:
data.head()

,input,output,category
"http://dbpedia.org/resource/Cathedral_of_Saint_Mary_of_the_Immaculate_Conception_(Peoria,_Illinois)","[[[135, 84, 44], [135, 84, 44], [135, 84, 44],...",1885.0,[http://dbpedia.org/resource/Gothic_Revival_ar...
http://dbpedia.org/resource/Eaton_Chapel,"[[[180, 100, 39], [180, 100, 39], [180, 100, 3...",1869.0,[http://dbpedia.org/resource/Gothic_Revival_ar...
http://dbpedia.org/resource/Santa_Maria_di_Costantinopoli,"[[[206, 227, 245], [198, 224, 242], [169, 202,...",1575.0,[http://dbpedia.org/resource/Baroque_architect...
"http://dbpedia.org/resource/St._Ambrose_Cathedral_(Des_Moines,_Iowa)","[[[242, 229, 207], [242, 229, 207], [243, 230,...",1890.0,[http://dbpedia.org/resource/Romanesque_Reviva...
"http://dbpedia.org/resource/St_Michael's_Church,_Baddiley","[[[215, 202, 200], [219, 206, 204], [223, 210,...",1308.0,[http://dbpedia.org/resource/English_Gothic_ar...


In [26]:
def get_all_activations(df, model):
    """
    Returns a DataFrame with all the information and activations associated
    :return: fully completed DataFrame
    """

    new_df = pd.DataFrame()

    new_df['category'] = df.category
    new_df['input'] = df.input
    """new_df['output_low'] = df.output_low
    new_df['output_medium'] = df.output_medium
    new_df['output_high'] = df.output_high"""
    new_df['output'] = df.output

    inputs = [x for x in df.input]

    activations = model.predict(inputs)

    # If the layer is an embedding layer, we take the mean of activations
    if isinstance(model.layers[-1], Embedding):

        emb_activations_arr = [a.flatten() for a in activations]
        emb_activations_arr = np.array(emb_activations_arr)

        acts = []
        for i in range(len(emb_activations_arr)):
            acts.append(pd.DataFrame(pd.DataFrame(emb_activations_arr[i].reshape(-1, 64)).mean()).T)

        activations = np.array(acts).reshape(-1, 64)

        for neuron_index, value_list in enumerate(activations.T):
            index = f"neuron_{neuron_index + 1}"
            new_df[index] = value_list


    else:
        for neuron_index, value_list in enumerate(activations.T):
            index = f"neuron_{neuron_index + 1}"
            new_df[index] = value_list

    #return_df = self.standardize(new_df)
    return new_df

In [33]:
inputs = [x for x in data.input]

In [87]:
new_model = Sequential()
act_list = []
for layer in model.layers:
    new_model.add(layer)
    print(f"Exploring layer {layer}")
    print(new_model.summary())
    activations = new_model.predict(inputs)
    act_list.append(activations)

Exploring layer <keras.layers.convolutional.conv2d.Conv2D object at 0x000002F64F28DA60>
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn1 (Conv2D)               (None, 193, 193, 8)       1544      
                                                                 
Total params: 1,544
Trainable params: 1,544
Non-trainable params: 0
_________________________________________________________________
None
16/16 [==============================] - 1s 74ms/step
Exploring layer <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000002F64E85AD90>
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn1 (Conv2D)               (None, 193, 193, 8)       1544      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 96, 8)     

In [42]:
for neuron_index, value_list in enumerate(act_list[0].T):
    print(neuron_index)
    print(len(value_list))
    print(value_list)

0
193
[[[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 ...

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]]
1
19

In [52]:
new_model.layers[0].output_shape[-1]

8

In [92]:
layer_to_explore = 3

for act in act_list[layer_to_explore]:
    a = act
    print(a.shape)
    if isinstance(model.layers[layer_to_explore], Conv2D) or isinstance(model.layers[layer_to_explore], MaxPooling2D): # same w/ embedding
        a = act.flatten()
        a = a.reshape(-1, new_model.layers[layer_to_explore].output_shape[-1])
    
pd.DataFrame(pd.DataFrame(a).mean()).T

(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 193, 8)
(193, 

ValueError: Must pass 2-d input. shape=(193, 193, 8)

In [86]:
model.layers[layer_to_explore].output_shape

(None, 73728)

In [83]:
a = a.flatten()
a = a.reshape(-1, 8)
pd.DataFrame(pd.DataFrame(a).mean()).T

,0,1,2,3,4,5,6,7
0,1.0,1.0,-1.0,-0.999946,-0.999848,-1.0,-1.0,-0.990396


In [28]:
get_all_activations(data, new_model)

16/16 [==============================] - 1s 79ms/step


ValueError: Length of values (193) does not match length of index (502)

In [2]:
df = pd.read_json('./data/painter_ds.json')

ValueError: Expected object or value